# Импорт

In [1]:
%load_ext tensorboard

In [2]:
import tensorflow_addons as tfa
from datetime import datetime
import numpy as np
import pandas as pd
import spacy
nlp = spacy.load("ru_core_news_sm")
import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
from json import load


c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# Формирование выборки

In [3]:
not_discriptors = list()
with open('../../Data/Words/word_pairs_not_disc.txt', 'r', encoding='utf-8') as txt:
    for i in txt:
        not_discriptors.append(i.replace('\n', ''))

with open('../../Data/Events/type_of_events.json', 'r', encoding='utf-8') as js:
    events = load(js)

with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discriptors = load(js)


In [4]:
dict_of_disc = {'expession': [], 'is_discr': []}
for subject in discriptors.values():
    for d in sum(subject.values(), []):
        dict_of_disc['expession'].append(nlp(d).vector)
        dict_of_disc['is_discr'].append(True)

for subject in not_discriptors:
    dict_of_disc['expession'].append(nlp(subject).vector)
    dict_of_disc['is_discr'].append(False)



In [5]:
features = np.array(dict_of_disc['expession'])
labels = np.array(list(map(int, dict_of_disc['is_discr'])))


In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=71)


# Создание модели 1

In [10]:
model_class1 = tf.keras.models.Sequential()
model_class1.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*1.5), activation='relu',input_shape=(X_train[0].shape[0],), name='input'))
model_class1.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*2), activation='relu', name='hidden_layer_2'))
# model_class1.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*3), activation='sigmoid', name='hidden_layer_3'))
# model_class1.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*2), activation='sigmoid', name='hidden_layer_4', use_bias=True))
# model_class1.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*1.5), activation='sigmoid', name='hidden_layer_5', use_bias=True))
# model_class1.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*0.5), activation='sigmoid', name='hidden_layer_6', use_bias=True))
model_class1.add(tf.keras.layers.Dense(1, name='output', activation='sigmoid'))
model_class1.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Dense)               (None, 144)               13968     
                                                                 
 hidden_layer_2 (Dense)      (None, 192)               27840     
                                                                 
 output (Dense)              (None, 1)                 193       
                                                                 


Total params: 42,001
Trainable params: 42,001
Non-trainable params: 0
_________________________________________________________________


# Обучение MSE

In [7]:
model_class1.compile(
    loss='mse',
    optimizer='Adam',
    metrics=[
        'mse',
    ]
)


In [8]:
model_class1.fit(
    X_train, 
    y_train, 
    validation_split=0.2, 
    epochs=20,
    batch_size=2
)


Epoch 1/20
319/319 [==============================] - 2s 3ms/step - loss: 0.1296 - mse: 0.1296 - val_loss: 0.0869 - val_mse: 0.0869
Epoch 2/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0758 - mse: 0.0758 - val_loss: 0.0575 - val_mse: 0.0575
Epoch 3/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0575 - mse: 0.0575 - val_loss: 0.0716 - val_mse: 0.0716
Epoch 4/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0553 - mse: 0.0553 - val_loss: 0.0675 - val_mse: 0.0675
Epoch 5/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0490 - mse: 0.0490 - val_loss: 0.0826 - val_mse: 0.0826
Epoch 6/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0644 - mse: 0.0644 - val_loss: 0.0555 - val_mse: 0.0555
Epoch 7/20
319/319 [==============================] - 1s 2ms/step - loss: 0.0351 - mse: 0.0351 - val_loss: 0.0647 - val_mse: 0.0647
Epoch 8/20
319/319 [==============================] - 1s 2ms/step - loss: 0.

In [9]:
predict = model_class1.predict(X_test)


9/9 [==============================] - 0s 2ms/step


In [10]:
mean_squared_error(predict, y_test)


0.046677894006748974

In [11]:
predictA = model_class1.predict(features)
print('Accuracy')
print(accuracy_score(labels, predictA > 0.5))
print('Confusion matrix')
print(confusion_matrix(labels, predictA > 0.5))
print('Precision, Recall, F\n', classification_report(
    labels, predictA > 0.5))


34/34 [==============================] - 0s 1ms/step
Accuracy
0.9699530516431925
Confusion matrix
[[334  24]
 [  8 699]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.98      0.93      0.95       358
           1       0.97      0.99      0.98       707

    accuracy                           0.97      1065
   macro avg       0.97      0.96      0.97      1065
weighted avg       0.97      0.97      0.97      1065



In [12]:
print('Accuracy')
print(accuracy_score(y_test, predict > 0.98))
print('Confusion matrix')
print(confusion_matrix(y_test, predict > 0.98))
print('Precision, Recall, F\n', classification_report(
    y_test, predict > 0.98))


Accuracy
0.3595505617977528
Confusion matrix
[[ 96   0]
 [171   0]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.36      1.00      0.53        96
           1       0.00      0.00      0.00       171

    accuracy                           0.36       267
   macro avg       0.18      0.50      0.26       267
weighted avg       0.13      0.36      0.19       267



c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [13]:
tn, fp, fn, tp = confusion_matrix(y_test, predict > 0.9).ravel()
(tn, fp, fn, tp)


(90, 6, 6, 165)

In [14]:
model_class1.predict(np.array([nlp('рекомендации, чтобы поддерживать внутреннюю').vector])), model_class1.predict(np.array([nlp('чтобы поддерживать внутреннюю').vector])), model_class1.predict(np.array([nlp('поддерживать внутреннюю структуру').vector]))



1/1 [==============================] - 0s 18ms/step


(array([[0.97134674]], dtype=float32),
 array([[0.02014221]], dtype=float32),
 array([[0.97330844]], dtype=float32))

# Обучение на Binary Crossentropy

In [7]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S") + '_BC'
tensorboard_callback_BC = tf.keras.callbacks.TensorBoard(log_dir=logdir)


In [11]:
model_class1.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer='Adam',
    metrics=[
        'categorical_crossentropy',
        tf.keras.metrics.FalsePositives(thresholds=0.5)
        # tfa.metrics.F1Score(num_classes=1, threshold=0.5)
    ]
)


In [12]:
model_class1.fit(
    X_train, 
    y_train, 
    validation_split=0.1, 
    epochs=26,
    batch_size=2,
    callbacks=[tensorboard_callback_BC],
)


Epoch 1/26
359/359 [==============================] - 4s 5ms/step - loss: 0.2750 - categorical_crossentropy: 0.0000e+00 - false_positives: 62.0000 - val_loss: 0.3197 - val_categorical_crossentropy: 0.0000e+00 - val_false_positives: 9.0000
Epoch 2/26
359/359 [==============================] - 1s 4ms/step - loss: 0.1310 - categorical_crossentropy: 0.0000e+00 - false_positives: 24.0000 - val_loss: 0.2759 - val_categorical_crossentropy: 0.0000e+00 - val_false_positives: 5.0000
Epoch 3/26
359/359 [==============================] - 1s 4ms/step - loss: 0.0745 - categorical_crossentropy: 0.0000e+00 - false_positives: 12.0000 - val_loss: 0.2252 - val_categorical_crossentropy: 0.0000e+00 - val_false_positives: 4.0000
Epoch 4/26
359/359 [==============================] - 1s 4ms/step - loss: 0.0212 - categorical_crossentropy: 0.0000e+00 - false_positives: 3.0000 - val_loss: 0.0955 - val_categorical_crossentropy: 0.0000e+00 - val_false_positives: 3.0000
Epoch 5/26
359/359 [=========================

In [13]:
%tensorboard --logdir logs/scalars/

In [14]:
predict = model_class1.predict(X_test)


9/9 [==============================] - 0s 3ms/step


In [15]:
mean_squared_error(predict, y_test)


0.03322374526140336

In [16]:
predictA = model_class1.predict(features)
print('Accuracy')
print(accuracy_score(labels, predictA > 0.5))
print('Confusion matrix')
print(confusion_matrix(labels, predictA > 0.5))
print('Precision, Recall, F\n', classification_report(
    labels, predictA > 0.5))


34/34 [==============================] - 0s 4ms/step
Accuracy
0.9859154929577465
Confusion matrix
[[348  10]
 [  5 702]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.99      0.97      0.98       358
           1       0.99      0.99      0.99       707

    accuracy                           0.99      1065
   macro avg       0.99      0.98      0.98      1065
weighted avg       0.99      0.99      0.99      1065



In [18]:
print('Accuracy')
print(accuracy_score(y_test, predict > 0.98))
print('Confusion matrix')
print(confusion_matrix(y_test, predict > 0.98))
print('Precision, Recall, F\n', classification_report(
    y_test, predict > 0.98))


Accuracy
0.9625468164794008
Confusion matrix
[[ 95   1]
 [ 40 131]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.95      0.95      0.95        96
           1       0.97      0.97      0.97       171

    accuracy                           0.96       267
   macro avg       0.96      0.96      0.96       267
weighted avg       0.96      0.96      0.96       267



In [21]:
tn, fp, fn, tp = confusion_matrix(y_test, predict > 0.9).ravel()
(tn, fp, fn, tp)


(91, 5, 6, 165)

## All

In [22]:
r = np.append(X_train, X_test, axis=0)
rr = np.append(y_train, y_test)

In [29]:
pred = model_class1.predict(r) > 0.5

34/34 [==============================] - 0s 1ms/step


In [30]:
print('Accuracy')
print(accuracy_score(rr, pred.T[0]))
print('Confusion matrix')
print(confusion_matrix(rr, pred.T[0]))
print('Precision, Recall, F\n', classification_report(
    rr, pred.T[0]))



Accuracy
0.9802816901408451
Confusion matrix
[[346  12]
 [  9 698]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.97      0.97      0.97       358
           1       0.98      0.99      0.99       707

    accuracy                           0.98      1065
   macro avg       0.98      0.98      0.98      1065
weighted avg       0.98      0.98      0.98      1065



## Example

In [ ]:
model_class1.predict(np.array([nlp('рекомендации, чтобы поддерживать внутреннюю').vector])), model_class1.predict(np.array([nlp('чтобы поддерживать внутреннюю').vector])), model_class1.predict(np.array([nlp('поддерживать внутреннюю структуру').vector]))



1/1 [==============================] - 0s 18ms/step


(array([[0.9891076]], dtype=float32),
 array([[0.9891076]], dtype=float32),
 array([[0.9891076]], dtype=float32))

# Обучение на Poison

In [ ]:
model_class1.compile(
    loss=tf.keras.losses.poisson,
    optimizer='Adam',
    metrics=[
        'mse',
    ]
)


In [ ]:
model_class1.fit(
    X_train, 
    y_train, 
    validation_split=0.2, 
    epochs=20,
    batch_size=2
)


Epoch 1/20
319/319 [==============================] - 2s 3ms/step - loss: 0.6954 - mse: 0.0111 - val_loss: 0.7846 - val_mse: 0.0504
Epoch 2/20
319/319 [==============================] - 1s 2ms/step - loss: 0.6922 - mse: 0.0078 - val_loss: 0.8116 - val_mse: 0.0556
Epoch 3/20
319/319 [==============================] - 1s 2ms/step - loss: 0.6905 - mse: 0.0062 - val_loss: 0.8132 - val_mse: 0.0556
Epoch 4/20
319/319 [==============================] - 1s 2ms/step - loss: 0.6904 - mse: 0.0062 - val_loss: 0.8174 - val_mse: 0.0557
Epoch 5/20
319/319 [==============================] - 1s 2ms/step - loss: 0.6905 - mse: 0.0062 - val_loss: 0.8150 - val_mse: 0.0557
Epoch 6/20
319/319 [==============================] - 1s 2ms/step - loss: 0.6905 - mse: 0.0062 - val_loss: 0.8143 - val_mse: 0.0557
Epoch 7/20
319/319 [==============================] - 1s 2ms/step - loss: 0.6905 - mse: 0.0062 - val_loss: 0.8141 - val_mse: 0.0557
Epoch 8/20
319/319 [==============================] - 1s 2ms/step - loss: 0.

In [ ]:
predict = model_class1.predict(X_test)


9/9 [==============================] - 0s 2ms/step


In [ ]:
mean_squared_error(predict, y_test)


0.0524771339190995

In [ ]:
predictA = model_class1.predict(features)
print('Accuracy')
print(accuracy_score(labels, predictA > 0.5))
print('Confusion matrix')
print(confusion_matrix(labels, predictA > 0.5))
print('Precision, Recall, F\n', classification_report(
    labels, predictA > 0.5))


34/34 [==============================] - 0s 2ms/step
Accuracy
0.9746478873239437
Confusion matrix
[[344  14]
 [ 13 694]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.96      0.96      0.96       358
           1       0.98      0.98      0.98       707

    accuracy                           0.97      1065
   macro avg       0.97      0.97      0.97      1065
weighted avg       0.97      0.97      0.97      1065



In [ ]:
print('Accuracy')
print(accuracy_score(y_test, predict > 0.98))
print('Confusion matrix')
print(confusion_matrix(y_test, predict > 0.98))
print('Precision, Recall, F\n', classification_report(
    y_test, predict > 0.98))


Accuracy
0.951310861423221
Confusion matrix
[[ 90   6]
 [  7 164]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.93      0.94      0.93        96
           1       0.96      0.96      0.96       171

    accuracy                           0.95       267
   macro avg       0.95      0.95      0.95       267
weighted avg       0.95      0.95      0.95       267



In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, predict > 0.9).ravel()
(tn, fp, fn, tp)


(89, 7, 7, 164)

In [ ]:
model_class1.predict(np.array([nlp('рекомендации, чтобы поддерживать внутреннюю').vector])), model_class1.predict(np.array([nlp('чтобы поддерживать внутреннюю').vector])), model_class1.predict(np.array([nlp('поддерживать внутреннюю структуру').vector]))



1/1 [==============================] - 0s 18ms/step


(array([[0.99341315]], dtype=float32),
 array([[0.9934132]], dtype=float32),
 array([[0.99341327]], dtype=float32))